In [ ]:
!pip install PyAthena[SQLAlchemy]
!aws configure get region
from sqlalchemy import create_engine
 
!pip install CurrencyConverter
import tr_dash_util as util
import numpy as np 
import html 
import json
import re
import pandas as pd
from datetime import datetime, timedelta
from currency_converter import CurrencyConverter, ECB_URL
from tr_dash_util import upload_dataframe_to_s3, extract_eReg_orders,clean_TR_dataframe,fetch_data_from_s3
from functools import reduce
import pytz


# set time
today=datetime.now()
today = today - timedelta(days=1)
 
# set env
prod=True

import pandas as pd

# Setting up the connection to Athena with the correct S3 staging directory
s3_staging_dir = "s3://ets-aws-plalab-dii-prod-analyticsbucket-1ktrlhzbrcbkb/athena_query_results/"
connection_string = f"awsathena+rest://:@athena.us-east-1.amazonaws.com:443/labsprodeventsdatabase-x806vjuzpbrd?s3_staging_dir={s3_staging_dir}"
engine = create_engine(connection_string)

# Query and data fetching
for day in range(1, 2):  # Only query for one previous day
    today = pd.Timestamp.now() - pd.Timedelta(days=day)
    year, month, day = today.strftime("%Y"), today.strftime("%m"), today.strftime("%d")

    query = f"""
    SELECT * FROM processed_events
    WHERE application_id='01845c7c-fa6d-4788-9ddd-cd888c977f36'
    AND year='{year}' AND month='{month}' AND day='{day}'
    AND env = 'prod' AND user_type = 'external_user'
    AND (event_name = 'ProfileCreated' OR event_name = 'OrderSubmitted')
    """
    
    # Fetching data using pandas
    ereg_full_df = pd.read_sql(query, engine)

    # Assuming util.clean_eReg_dataframe is a function you've defined to clean your dataframe
    ereg_full_df_clean = util.clean_eReg_dataframe(ereg_full_df)

In [ ]:
import datetime
#from datetime import timedelta
#import pandas as pd
 
# Function to generate date range
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days) + 1):
        yield start_date + timedelta(n)



# Function to get target data        
def get_target(cleaned_ereg_df):
    # Filter for 'Profile Created' events
    profile_created = cleaned_ereg_df[cleaned_ereg_df['event_name'] == 'ProfileCreated']
    # Get the first registration time for each user
    first_registration_time = profile_created.groupby('user_id')['event_timestamp'].min().reset_index()
    first_registration_time.rename(columns={'event_timestamp': 'time_new_user_first_rgst_account'}, inplace=True)
    # Filter for 'Order Submitted' events
    order_submitted = cleaned_ereg_df[cleaned_ereg_df['event_name'] == 'OrderSubmitted']
    # Get the first order submission time for each user
    first_order_time = order_submitted.groupby('user_id')['event_timestamp'].min().reset_index()
    first_order_time.rename(columns={'event_timestamp': 'time_user_order_submitted'}, inplace=True)
    # Merge the two dataframes on user_id
    df_target = pd.merge(first_registration_time, first_order_time, on='user_id', how='inner')
    # Calculate the time difference in seconds (assuming timestamp is in microseconds)
    df_target['time_diff(sec)'] = (df_target['time_user_order_submitted'] - df_target['time_new_user_first_rgst_account']) / 1e6
    return df_target
 

start_date = datetime.datetime(2024, 2, 1)
end_date = datetime.datetime(2024, 2, 10)
 
data_frames = []  # To collect data frames from each day
 
for single_date in daterange(start_date, end_date):
    year, month, day = single_date.strftime("%Y"), single_date.strftime("%m"), single_date.strftime("%d")
    print(f"Fetching data for {single_date.strftime('%Y-%m-%d')}...")
 
    query = f"""
    SELECT * FROM processed_events
    WHERE application_id='01845c7c-fa6d-4788-9ddd-cd888c977f36'
    AND year='{year}' AND month='{month}' AND day='{day}'
    AND env = 'prod' AND user_type = 'external_user'
    AND (event_name = 'ProfileCreated' OR event_name = 'OrderSubmitted')
    """
    ereg_full_df = pd.read_sql(query, engine)
    ereg_full_df_clean = util.clean_eReg_dataframe(ereg_full_df)
    data_frames.append(ereg_full_df_clean)
    

    
#concatenate all dataframes into one
full_data_frame = pd.concat(data_frames, ignore_index=True)
 
# Calculate the target data
df_target = get_target(full_data_frame)
print(df_target)